In [9]:
import numpy as np
import gmsh

In [10]:
densidad= 7850 #
A=10E-4
L_total=1
I=10E-8
E= 210E9
numero_elementos=3
numero_nodos=numero_elementos+1

In [11]:
matriz_nodos=np.linspace(0,L_total,numero_elementos+1).reshape(-1,1)
matriz_nodos=np.hstack([matriz_nodos , np.zeros_like(matriz_nodos)])
print(matriz_nodos)
matriz_conectividad=np.array([[0,1],[1,2],[2,3]])
matriz_global_rigidez=np.zeros((2*numero_nodos,2*numero_nodos))
matriz_global_concentrada=np.zeros((2*numero_nodos,2*numero_nodos))
L=L_total/numero_elementos
s=[0,1]
r=(np.linspace(2,2*numero_nodos-1,2*numero_nodos-2)).astype(int)
print(r)

[[0.         0.        ]
 [0.33333333 0.        ]
 [0.66666667 0.        ]
 [1.         0.        ]]
[2 3 4 5 6 7]


In [12]:
for i in range (len(matriz_conectividad)):
    (a),(b)=matriz_conectividad[i] 
    a=int(a)
    b=int(b)
    x1=matriz_nodos[a,0]
    x2=matriz_nodos[b,0]
    y1=matriz_nodos[a,1]
    y2=matriz_nodos[b,1]

    coef_k=E*I/L**3
    k=coef_k*np.array([[12,6*L,-12,6*L],[6*L,4*L**2,-6*L,2*L**2],[-12,-6*L,12,-6*L],[6*L,2*L**2,-6*L,4*L**2]])
    matriz_global_rigidez[2*a:2*(a+1), 2*a:2*(a+1)] += k [0:2,0:2]
    matriz_global_rigidez[2*(a):2*(a+1), 2*(b):2*(b+1)] += k [0:2,2:4]
    matriz_global_rigidez[2*b:2*(b+1), 2*b:2*(b+1)] += k [2:4,2:4]
    matriz_global_rigidez[2*b:2*(b+1), 2*a:2*(a+1)] += k [2:4,0:2]

    matriz_concentrada=densidad*A*(L/24)*np.array([[12,0,0,0],[0,L**2,0,0],[0,0,12,0],[0,0,0,L**2]])
    matriz_global_concentrada[2*a:2*(a+1), 2*a:2*(a+1)] += matriz_concentrada [0:2,0:2]
    matriz_global_concentrada[2*(a):2*(a+1), 2*(b):2*(b+1)] += matriz_concentrada [0:2,2:4]
    matriz_global_concentrada[2*b:2*(b+1), 2*b:2*(b+1)] += matriz_concentrada [2:4,2:4]
    matriz_global_concentrada[2*b:2*(b+1), 2*a:2*(a+1)] += matriz_concentrada [2:4,0:2]
#print(matriz_global_rigidez)
#print()
#print(matriz_global_concentrada)
#scale=np.max((np.max(matriz_global)))
#matriz_global[abs(matriz_global/scale) < 1e-9] = 0

In [13]:
print(matriz_global_concentrada[np.ix_(r,r)].shape, matriz_global_rigidez[np.ix_(r,r)].shape)

(6, 6) (6, 6)


In [14]:
from scipy.linalg import eigh 
w, vl_r = eigh(matriz_global_rigidez[np.ix_(r,r)], matriz_global_concentrada[np.ix_(r,r)])
print(vl_r)

[[ 0.1078782   0.32978952 -0.44635954  0.23469896  0.06939118 -0.05129516]
 [ 0.59378168  1.13877527  0.10705238 -3.45014763  5.12059322 -1.2202646 ]
 [ 0.36148961  0.3421096   0.26434642 -0.10119412 -0.21382409 -0.09280017]
 [ 0.88156788 -1.30291596  2.04871432  3.90822572  1.82482791 -3.99970973]
 [ 0.67109076 -0.42334539 -0.14405979  0.04403293  0.12318331  0.31125687]
 [ 0.9537416  -2.90348891 -3.70944928 -3.72463733 -3.30034417 -5.88920122]]


In [17]:

vl=np.vstack([np.zeros((2,2*(numero_nodos-1))),vl_r])
print(vl_r.shape)
print(vl)

(6, 6)
[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.1078782   0.32978952 -0.44635954  0.23469896  0.06939118 -0.05129516]
 [ 0.59378168  1.13877527  0.10705238 -3.45014763  5.12059322 -1.2202646 ]
 [ 0.36148961  0.3421096   0.26434642 -0.10119412 -0.21382409 -0.09280017]
 [ 0.88156788 -1.30291596  2.04871432  3.90822572  1.82482791 -3.99970973]
 [ 0.67109076 -0.42334539 -0.14405979  0.04403293  0.12318331  0.31125687]
 [ 0.9537416  -2.90348891 -3.70944928 -3.72463733 -3.30034417 -5.88920122]]


In [ ]:
frecuencia=np.sqrt(w)/(2*np.pi)
print(frecuencia)

In [18]:
vl_normalizado=vl/vl[2*numero_nodos-2,:]
print(vl_normalizado)

[[  0.          -0.          -0.           0.           0.
    0.        ]
 [  0.          -0.          -0.           0.           0.
    0.        ]
 [  0.16075054  -0.77900818   3.09843243   5.33007887   0.56331641
   -0.1648001 ]
 [  0.88480086  -2.68994372  -0.74311074 -78.35381374  41.56888739
   -3.92044236]
 [  0.53865979  -0.80810989  -1.83497709  -2.29814666  -1.73582027
   -0.29814659]
 [  1.31363436   3.07766659 -14.22127759  88.75689478  14.81392144
  -12.85018959]
 [  1.           1.           1.           1.           1.
    1.        ]
 [  1.42118125   6.85843991  25.74937235 -84.58755117 -26.79213695
  -18.92071111]]
